#  台标识别
### 准备数据集

从s3上下载压缩文件， 解压到./data/tv-icon-dataset-original 文件夹下

In [1]:
import urllib 

urllib.request.urlretrieve('https://s3.amazonaws.com/dikers.nwcd/data-set/tv-icon-dataset-original.zip', "tv-icon-dataset-original.zip")

('tv-icon-dataset-original.zip', <http.client.HTTPMessage at 0x7fa9781ae290>)

In [2]:
!unzip  ./tv-icon-dataset-original.zip
!rm ./tv-icon-dataset-original.zip
!mv tv-icon-dataset-original ./data/tv-icon-dataset-original
!mkdir ./data/target -p

Archive:  ./tv-icon-dataset-original.zip
   creating: tv-icon-dataset-original/
   creating: tv-icon-dataset-original/cctv13/
  inflating: tv-icon-dataset-original/cctv13/016.JPG  
  inflating: tv-icon-dataset-original/cctv13/002.JPG  
  inflating: tv-icon-dataset-original/cctv13/003.JPG  
  inflating: tv-icon-dataset-original/cctv13/017.JPG  
  inflating: tv-icon-dataset-original/cctv13/149.JPG  
  inflating: tv-icon-dataset-original/cctv13/029.JPG  
  inflating: tv-icon-dataset-original/cctv13/001.JPG  
  inflating: tv-icon-dataset-original/cctv13/015.JPG  
  inflating: tv-icon-dataset-original/cctv13/014.JPG  
  inflating: tv-icon-dataset-original/cctv13/028.JPG  
  inflating: tv-icon-dataset-original/cctv13/148.JPG  
  inflating: tv-icon-dataset-original/cctv13/158.JPG  
  inflating: tv-icon-dataset-original/cctv13/.DS_Store  
   creating: __MACOSX/tv-icon-dataset-original/
   creating: __MACOSX/tv-icon-dataset-original/cctv13/
  inflating: __MACOSX/tv-icon-dataset-original/cctv13/

In [106]:
!wget http://dikers.nwcd.s3-website-us-east-1.amazonaws.com/data-set/tv-icon-dataset.zip
!mv ./tv-icon-dataset.zip ./data/
!unzip  ./data/tv-icon-dataset.zip
!rm ./data/tv-icon-dataset.zip

--2019-11-07 16:00:27--  http://dikers.nwcd.s3-website-us-east-1.amazonaws.com/data-set/tv-icon-dataset.zip
Resolving dikers.nwcd.s3-website-us-east-1.amazonaws.com (dikers.nwcd.s3-website-us-east-1.amazonaws.com)... 52.216.104.226
Connecting to dikers.nwcd.s3-website-us-east-1.amazonaws.com (dikers.nwcd.s3-website-us-east-1.amazonaws.com)|52.216.104.226|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2179161 (2.1M) [application/zip]
Saving to: ‘tv-icon-dataset.zip.1’

tv-icon-dataset.zip  19%[==>                 ] 417.82K  3.83KB/s    eta 4m 37s ^C
Archive:  ./data/tv-icon-dataset.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of ./data/tv-icon-dataset.zip or
        ./data/tv-icon-dataset.zip.zip, and canno

In [137]:
from keras import optimizers
from keras.models import Sequential
from keras.layers import Conv2D, Dense, Flatten, MaxPooling2D, Dropout
from keras.callbacks import LearningRateScheduler, TensorBoard
import cv2
import os
from keras.preprocessing.image import img_to_array
from sklearn import model_selection
import numpy as np


In [138]:
# 获取所有图片，制作数据集
def get_file_list(file_dir):
    """
    :获取文件列表
    :param file_dir:
    :return:
    """
    L = []
    for root, dirs, files in os.walk(file_dir):
        for file in files:
            if os.path.splitext(file)[1] == '.JPG':
                input_file_name = os.path.join(root, file)
                L.append(input_file_name)
    return L



In [139]:

def load_data(dir_list):

    images = []
    labels = []

    index = 0
    for path_dir in dir_list:
        L = get_file_list(path_dir)

        for image_path in L:
            image = cv2.imread(image_path)
            image = cv2.resize(image, (IMAGE_DIMS[1], IMAGE_DIMS[0]))
            image = img_to_array(image)
            images.append(image)
            label_item = []
            for i in range(len(dir_list)):
                label_item.append(0)
            label_item[index] = 1 # 设置label

            labels.append(label_item)

        index += 1

    images = np.array(images, dtype='float')
    labels = np.array(labels, dtype='int')
    x_train, x_test, y_train , y_test= \
        model_selection.train_test_split(images, labels, test_size=0.2)

    print("================================== ")
    print('x_train: ' ,x_train.shape)
    print('x_test : ' ,x_test.shape)
    print('y_train: ' ,y_train.shape)
    print('y_test : ' ,y_test.shape)


    return x_train, x_test, y_train, y_test


In [140]:
def get_dir_list(file_dir):
    """
    :获取目录的list
    :param file_dir:
    :return:
    """
    L = []
    for root, dirs, files  in os.walk(file_dir):
        for item in dirs:
            L.append(file_dir+item)
    return L

In [141]:
dir_list = get_dir_list('./data/tv-icon-dataset-original/')
# dir_list = get_dir_list('./data/train/')
class_count = len(dir_list)
print(dir_list)

['./data/tv-icon-dataset-original/cctv13', './data/tv-icon-dataset-original/cctv4', './data/tv-icon-dataset-original/cctv1']


In [142]:
# load data
x_train, x_test, y_train, y_test = load_data(dir_list)

x_train:  (461, 140, 140, 3)
x_test :  (116, 140, 140, 3)
y_train:  (461, 3)
y_test :  (116, 3)


In [143]:
IMAGE_DIMS = (140, 140, class_count)

def build_model():
    model = Sequential()
    model.add(Conv2D(6, (5, 5), padding='valid', activation = 'relu', kernel_initializer='he_normal', input_shape=IMAGE_DIMS))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    model.add(Conv2D(16, (5, 5), padding='valid', activation = 'relu', kernel_initializer='he_normal'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    model.add(Flatten())
    model.add(Dropout(0.5))
    model.add(Dense(120, activation = 'relu', kernel_initializer='he_normal'))
    model.add(Dropout(0.5))
    model.add(Dense(84, activation = 'relu', kernel_initializer='he_normal'))
    model.add(Dense(class_count, activation = 'softmax', kernel_initializer='he_normal'))
    sgd = optimizers.SGD(lr=.1, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
    return model


In [144]:
def scheduler(epoch):
    if epoch < 10:
        return 0.01
    if epoch < 20:
        return 0.002
    return 0.01

In [145]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255.0
x_test /= 255.0


# build network
model = build_model()
print(model.summary())

# set callback
tb_cb = TensorBoard(log_dir='./data/target', histogram_freq=0)
change_lr = LearningRateScheduler(scheduler)
cbks = [change_lr,tb_cb]

# start train
model.fit(x_train, y_train,
          batch_size=64,
          epochs=10,
          callbacks=cbks,
          validation_data=(x_test, y_test),
          shuffle=True)

# save model
model.save('./data/target/lenet.h5')

result = model.predict(x_test)

total_count = len(result)

right_count = 0
for pre, real in zip(result, y_test):
    # print(pre, real)
    if np.argmax(pre) == np.argmax(real):
        right_count += 1
print('socre {} '.format(right_count * 100 / total_count))

W1107 16:07:23.853793 140063343695680 deprecation.py:506] From /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_31 (Conv2D)           (None, 136, 136, 6)       456       
_________________________________________________________________
max_pooling2d_31 (MaxPooling (None, 68, 68, 6)         0         
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 64, 64, 16)        2416      
_________________________________________________________________
max_pooling2d_32 (MaxPooling (None, 32, 32, 16)        0         
_________________________________________________________________
flatten_16 (Flatten)         (None, 16384)             0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16384)             0         
_________________________________________________________________
dense_43 (Dense)             (None, 120)               1966200   
__________